In [ ]:
from pathlib import Path
import glob
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import copy

import open3d as o3d
import nibabel as nib
from skimage.measure import marching_cubes


In [ ]:
DATA_ROOT = Path("C:/Users/aorhu/Masaüstü/samples")
TARGET_ROOT = "D:/decimated"
VERTICES = 25000
EXTENSION = ".ply"
LIMIT = 10000
COUNT = 0

In [ ]:
def process(DATA_ROOT, TARGET_ROOT, VERTICES, EXTENSION, LIMIT, COUNT):

    for file in tqdm(os.listdir(str(DATA_ROOT))):
        _path = str(os.path.join(str(DATA_ROOT), file).replace('\\', '/')) +'/body_mask.nii.gz'
    
        _id = _path[_path[:_path.rfind("/")].rfind("/")+1:_path.rfind("/",0,)]
        
        if(os.path.exists(_path) and (LIMIT > COUNT) ):
            
            body_segment = nib.load(_path)
            body_segment_data = body_segment.get_fdata()

            verts, faces, _, __ = marching_cubes(body_segment_data, level=0, step_size=1)
            verts = verts/np.array(body_segment_data.shape) 

            mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                        triangles=o3d.utility.Vector3iVector(np.asarray(faces)))

            decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, VERTICES)
            
            if (len(decimated_mesh.triangles) == VERTICES):
                _target_path =  TARGET_ROOT + "/" + _id + EXTENSION
                print( _target_path)
                o3d.io.write_triangle_mesh( _target_path, decimated_mesh)
                print('\r' + str(COUNT) + "written", end='')
                COUNT += 1
        else:
            continue

In [ ]:
process(DATA_ROOT, TARGET_ROOT, VERTICES, EXTENSION, LIMIT, COUNT)

In [ ]:
TARGET_ROOT = Path("D:/decimated")

In [ ]:
for file in glob.glob(str(TARGET_ROOT / "*.ply")):
    _path = file.replace('\\', '/')
    print(_path)

In [ ]:
def draw_registration_result(source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [ ]:
source = o3d.io.read_point_cloud("D:/decimated/1000071.ply")
target = o3d.io.read_point_cloud("D:/decimated/1000125.ply")
threshold = 0.02

draw_registration_result(source, target)

In [ ]:
source.estimate_normals()
target.estimate_normals()

In [ ]:
threshold = 0.02

draw_registration_result(source, target)

In [ ]:
trans_init = np.identity(4)

In [ ]:
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init, 
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

In [ ]:
reg_p2l.transformation

In [ ]:
source_temp = copy.deepcopy(source)

In [ ]:
source_temp.transform(reg_p2l.transformation)

In [ ]:
np.asarray(source.points)

In [ ]:
np.asarray(source_temp.points)

In [ ]:
o3d.visualization.draw_geometries([source_temp, source])

In [ ]:
sample_pcd = o3d.io.read_point_cloud("D:/ADLM_Data/icp_target/4266049_5.ply")

In [ ]:
o3d.visualization.draw_geometries([sample_pcd])

### reading as triangle mesh, converting to pcd, apply icp, convert to triangle mesh and write a triangle mesh

In [ ]:
target_trm = o3d.io.read_triangle_mesh("D:/ADLM_Data/icp_target/4266049_25.ply")
target_trm.compute_vertex_normals()
target_pcd = o3d.geometry.PointCloud(points = target_trm.vertices)
target_pcd.estimate_normals()

In [ ]:
o3d.visualization.draw_geometries([target_trm],mesh_show_back_face=True,mesh_show_wireframe=True)

In [ ]:
def draw_registration_result(source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [ ]:
source_trm = o3d.io.read_triangle_mesh("D:/ADLM_Data/decimated/1000071.ply")

In [ ]:
source_trm

#### convert to pcd

In [ ]:
source_pcd = o3d.geometry.PointCloud(points = source_trm.vertices)
source_pcd.estimate_normals()

In [ ]:
threshold = 0.02
trans_init = np.identity(4)

In [ ]:
reg_p2l = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd, threshold, trans_init, 
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

In [ ]:
reg_p2l.transformation

In [ ]:
source_temp = copy.deepcopy(source_pcd)

In [ ]:
source_temp.transform(reg_p2l.transformation)

In [ ]:
np.asarray(source_pcd.points)


In [ ]:
np.asarray(source_temp.points)

In [ ]:
o3d.visualization.draw_geometries([source_temp, source_pcd])

In [ ]:
new_mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(source_temp.points)),
                                    triangles=o3d.utility.Vector3iVector(np.asarray(source_trm.triangles)))
new_mesh.compute_vertex_normals()

In [ ]:
original_mesh_paint = np.asarray([0,200,220])/255.0
registered_mesh_paint = np.asarray([230,200,110])/255.0
source_trm.paint_uniform_color(original_mesh_paint)
new_mesh.paint_uniform_color(registered_mesh_paint)

In [ ]:
source_trm.compute_vertex_normals()
new_mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([source_trm,new_mesh],mesh_show_back_face=True,mesh_show_wireframe=True)